# Linkedin

**Trabajos.**

https://www.linkedin.com/jobs/search/?geoId=105646813&keywords=data%20science&location=Espa%C3%B1a

**Salarios.**

https://www.linkedin.com/salary/explorer?countryCode=es&geoId=105646813&titleId=340

## Plots